In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import tensorflow as tf

import image_embedding
import image_processing
import inputs as input_ops

import numpy as np
import utils_project 
#import utils_caltech
import configuration
import cifar10
from cifar10 import img_size, num_channels, num_classes,_num_images_train

In [27]:
print(tf.__version__)

0.12.0


In [4]:
def random_batch(image_train,train_batch_size):
    # Number of images in the training-set.
    num_images = len(images_train)

    # Create a random index.
    idx = np.random.choice(num_images,
                           size=train_batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    x_batch = images_train[idx, :, :, :]
    y_batch = labels_train[idx, :]

    return x_batch, y_batch

In [5]:
def predict_cls(images, labels, cls_true):
    # Number of images.
    num_images = len(images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_images, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_images:
        # The ending index for the next batch is denoted j.
        j = min(i + batch_size, num_images)

        # Create a feed-dict with the images and labels
        # between index i and j.
        feed_dict = {x: images[i:j, :],
                     y_true: labels[i:j, :]}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    return correct, cls_pred

In [6]:
#tf.reset_default_graph()

utils = utils_project

height = 32
width = 32

X = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

with tf.name_scope("convol_dnn"):
    
    #shape formula W2=(W1−F+2P)/S+1 
    num_input_channels=3
    num_inputs = (32*32*3)
    num_filters = 64
    kernel_size = 5
    layern1 = utils.conv_layer(X,"layer1",
                                       use_pooling=False,
                                       dropout = False,
                                       activation="leak_relu",
                                       num_inputs=num_inputs,
                                       num_input_channels=num_input_channels,
                                       kernel_size=kernel_size,
                                       num_filters=num_filters,
                                       padding = 'SAME')
    
    #num_inputs = 32*32*64
    #pool formula (W1−F)/S+1  
    #pool shape from utils_project.conv_layer ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1]
    layern2 = utils_project.conv_layer(layern1,"layer2",use_pooling=True,
                                       dropout=True,
                                       activation="leak_relu",
                                       num_inputs=32*32*64,
                                       num_input_channels=64,
                                       kernel_size=5,
                                       num_filters=128,
                                       padding = 'SAME')
    #output shape [none, 16,16,128]
    
    
    layern3 = utils_project.conv_layer(layern2,"layer3",use_pooling=True,
                                       dropout=True,
                                       activation="leak_relu",
                                       num_inputs=16*16*128,
                                       num_input_channels=128,
                                       kernel_size=5,
                                       num_filters=328,
                                       padding = 'SAME')
    #output shape [none, 8,8,328]
    
    #[None,8*8*328]
    flatten_tensor,num_features = utils_project.flatten_layer(layern3)
    
    layer_fc1 = utils_project.new_fc_layer(input=flatten_tensor,
                         num_inputs=num_features,
                         num_outputs=328,
                         activation="leak_relu")
    
    layer_fc2 = utils_project.new_fc_layer(input=layer_fc1,
                         num_inputs=328,
                         num_outputs=10,
                         activation="leak_relu")

    
    y_pred = tf.nn.softmax(layer_fc2)
    
    y_pred_cls = tf.argmax(y_pred, dimension=1)
    
    
with tf.name_scope("loss"):
    print(layer_fc2)
    #xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(layer_fc2, y_true)
    xentropy = tf.nn.softmax_cross_entropy_with_logits(layer_fc2, y_true)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #correct = tf.nn.in_top_k(logits, y, 1)
    #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()    
   

Tensor("convol_dnn/leak_relu_1:0", shape=(?, 10), dtype=float32)


In [ ]:
n_epochs = 1
batch_size = 500

#load dataset 


images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()



print(_num_images_train)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(1):
            X_batch, y_batch = random_batch(images_train,batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y_true: y_batch})
            # Print status every 100 iterations.
            if i % 100 == 0:
                # Calculate the accuracy on the training-set.
                acc_train = accuracy.eval(feed_dict={X: X_batch, y_true: y_batch})
                # Message for printing.
                msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

                # Print it.
                print(msg.format(i + 1, acc_train))
            
    
        acc_test = accuracy.eval(feed_dict={X: images_test, y_true: labels_test})
        #print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "my_model_final.ckpt")

Loading data: data/cifar-10-batches-py/data_batch_1
Loading data: data/cifar-10-batches-py/data_batch_2
Loading data: data/cifar-10-batches-py/data_batch_3
Loading data: data/cifar-10-batches-py/data_batch_4
Loading data: data/cifar-10-batches-py/data_batch_5
Loading data: data/cifar-10-batches-py/test_batch
50000
0
0.136
